# Notebook 05 – Baseline Models (TF-IDF + Similarité Cosine)

## Objectifs
- Construire un système de recommandation basique "content-based"
- Évaluer les performances avec des métriques adaptées (Precision@K, Recall@K)
- Créer une base de comparaison pour les modèles ML/DL à venir


### 1. Chargement des données et prétraitements

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import joblib

# Chargement des données
df = pd.read_csv("../data/processed/anime_clean.csv")
features = joblib.load("../data/processed/final_features_sparse.pkl")

print(f"Shape des features: {features.shape}")
print(f"Shape du dataframe: {df.shape}")


### 2. Calcul de la similarité de contenu

On part d’un simple modèle de similarité cosinus entre les animes.

In [ ]:
# Calcul de la matrice de similarité (approximation possible sur 5000 premiers si lourd)
sample_size = min(5000, features.shape[0])
similarity_matrix = cosine_similarity(features[:sample_size])

print(f"✅ Matrice de similarité calculée: {similarity_matrix.shape}")


### 3.Fonction de recommandation

In [ ]:
def get_recommendations(title, top_n=10):
    """Retourne les animes les plus similaires à un titre donné"""
    if title not in df['title'].values:
        return f"⚠️ '{title}' non trouvé dans la base."

    idx = df.index[df['title'] == title][0]
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]

    anime_indices = [i[0] for i in sim_scores]
    recommendations = df.iloc[anime_indices][['title', 'score', 'genres', 'type', 'year']]
    recommendations['similarity'] = [i[1] for i in sim_scores]
    return recommendations.reset_index(drop=True)


### 4. Test du modèle

In [ ]:
# Exemple de test
example_title = df.iloc[100]['title']
print(f"🎬 Recommandations pour : {example_title}")
recs = get_recommendations(example_title, top_n=5)
display(recs)


### 5.Évaluation Baseline – Métriques de recommandation

In [ ]:
from sklearn.metrics import precision_score, recall_score

def precision_at_k(recommended, relevant, k):
    return len(set(recommended[:k]) & set(relevant)) / k

def recall_at_k(recommended, relevant, k):
    return len(set(recommended[:k]) & set(relevant)) / len(relevant)

# Exemple simulé :
relevant_items = set(df[df['genres'].str.contains('Action', na=False)].sample(10)['title'])
recommended_items = set(get_recommendations(example_title, top_n=10)['title'])

print(f"Precision@10 = {precision_at_k(list(recommended_items), list(relevant_items), 10):.2f}")
print(f"Recall@10 = {recall_at_k(list(recommended_items), list(relevant_items), 10):.2f}")


### 6. Visualisation des similarités

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

top_sim = similarity_matrix[idx][:20]
titles = df['title'][:20]

plt.figure(figsize=(10, 6))
sns.barplot(x=top_sim, y=titles)
plt.title(f"Top similarités pour {example_title}")
plt.xlabel("Score de similarité")
plt.show()


### 7. Sauvegarde et préparation du modèle pour Streamlit

In [ ]:
import pickle

# Sauvegarde de la matrice de similarité
pickle.dump(similarity_matrix, open("../models/baseline_similarity.pkl", "wb"))
print("✅ Modèle baseline sauvegardé avec succès !")


📘 Résumé

In [ ]:
## 📘 Résumé - Baseline Models

| Élément | Méthode | Détails |
|----------|----------|----------|
| Type de modèle | Content-Based | TF-IDF + Cosine Similarity |
| Données utilisées | features_engineered.csv | Texte, numériques, catégorielles |
| Métriques simulées | Precision@K, Recall@K | (User profiling à venir) |
| Points forts | Simple, interprétable, rapide |
| Limites | Pas de personnalisation utilisateur |
| Suivant | Notebook 06 – Machine Learning (RandomForest, XGBoost) |
